In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

from fastai2.layers import LabelSmoothingCrossEntropy
from transformers import get_cosine_schedule_with_warmup
from transformers import AdamW

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "02_starter_code"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 32
    model_name = 'efficientnet-b0'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  02_starter_code  already exists


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.OneOf([
            A.Rotate(limit=90, p=1),
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.OpticalDistortion(p=1),
            A.ShiftScaleRotate(rotate_limit=90, shift_limit=0.2, scale_limit=0.1, p=1)
        ], p=0.8),
        
        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.RandomContrast(limit=0.2, p=1),
            A.ElasticTransform(p=1),
            A.IAAPiecewiseAffine(p=1)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.2, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
    hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
    hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b0


In [8]:
criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)

num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

In [9]:
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True, mode="min", factor=0.3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

# end_lr = 0.1
# start_lr = 1e-7
# lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (lr_find_epochs * len(train_dataloader)))
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# scheduler_1 = LinearCyclicalScheduler(optimizer, "lr", start_value=0.01, end_value=0.005, cycle_size=60)
# scheduler_2 = CosineAnnealingScheduler(optimizer, "lr", start_value=0.05, end_value=0.001, cycle_size=60)
# combined_scheduler = ConcatScheduler(schedulers=[scheduler_1, scheduler_2], durations=[30, ])

In [10]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_loss={best_valid_loss}.pth")
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [12]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 14s
Training Results - Average Loss: 1.3997 | auc/_mean: 0.4781 | auc/healthy: 0.4465 | auc/multiple_diseases: 0.4846, auc/rust: 0.4743, auc/scab: 0.5072
Evaluating Results - Average Loss: 1.3781 | auc/_mean: 0.4590 | auc/healthy: 0.5229 | auc/multiple_diseases: 0.5535, auc/rust: 0.3128, auc/scab: 0.4469




Epoch: 02 | Time: 0m 13s
Training Results - Average Loss: 1.4012 | auc/_mean: 0.4767 | auc/healthy: 0.4488 | auc/multiple_diseases: 0.5195, auc/rust: 0.4339, auc/scab: 0.5044
Evaluating Results - Average Loss: 1.3748 | auc/_mean: 0.4555 | auc/healthy: 0.4641 | auc/multiple_diseases: 0.5548, auc/rust: 0.3485, auc/scab: 0.4546




Epoch: 03 | Time: 0m 14s
Training Results - Average Loss: 1.3841 | auc/_mean: 0.4974 | auc/healthy: 0.4902 | auc/multiple_diseases: 0.5232, auc/rust: 0.4574, auc/scab: 0.5187
Evaluating Results - Average Loss: 1.3698 | auc/_mean: 0.4490 | auc/healthy: 0.4113 | auc/multiple_diseases: 0.5053, auc/rust: 0.3858, auc/scab: 0.4937




Epoch: 04 | Time: 0m 14s
Training Results - Average Loss: 1.3705 | auc/_mean: 0.4975 | auc/healthy: 0.4697 | auc/multiple_diseases: 0.4649, auc/rust: 0.5183, auc/scab: 0.5371
Evaluating Results - Average Loss: 1.3612 | auc/_mean: 0.4505 | auc/healthy: 0.3978 | auc/multiple_diseases: 0.4488, auc/rust: 0.4164, auc/scab: 0.5392




Epoch: 05 | Time: 0m 14s
Training Results - Average Loss: 1.3666 | auc/_mean: 0.4999 | auc/healthy: 0.5004 | auc/multiple_diseases: 0.4860, auc/rust: 0.4763, auc/scab: 0.5368
Evaluating Results - Average Loss: 1.3497 | auc/_mean: 0.4656 | auc/healthy: 0.4149 | auc/multiple_diseases: 0.4256, auc/rust: 0.4505, auc/scab: 0.5716



...


Epoch: 46 | Time: 0m 14s
Training Results - Average Loss: 1.1996 | auc/_mean: 0.6753 | auc/healthy: 0.6701 | auc/multiple_diseases: 0.5559, auc/rust: 0.7286, auc/scab: 0.7465
Evaluating Results - Average Loss: 1.1536 | auc/_mean: 0.7045 | auc/healthy: 0.7425 | auc/multiple_diseases: 0.3963, auc/rust: 0.8246, auc/scab: 0.8546




Epoch: 47 | Time: 0m 13s
Training Results - Average Loss: 1.2000 | auc/_mean: 0.6804 | auc/healthy: 0.6758 | auc/multiple_diseases: 0.5654, auc/rust: 0.7371, auc/scab: 0.7435
Evaluating Results - Average Loss: 1.1507 | auc/_mean: 0.7063 | auc/healthy: 0.7476 | auc/multiple_diseases: 0.3934, auc/rust: 0.8282, auc/scab: 0.8561




Epoch: 48 | Time: 0m 14s
Training Results - Average Loss: 1.2145 | auc/_mean: 0.6408 | auc/healthy: 0.6623 | auc/multiple_diseases: 0.4475, auc/rust: 0.7135, auc/scab: 0.7399
Evaluating Results - Average Loss: 1.1465 | auc/_mean: 0.7106 | auc/healthy: 0.7548 | auc/multiple_diseases: 0.3935, auc/rust: 0.8341, auc/scab: 0.8600




Epoch: 49 | Time: 0m 14s
Training Results - Average Loss: 1.1927 | auc/_mean: 0.6767 | auc/healthy: 0.6968 | auc/multiple_diseases: 0.5179, auc/rust: 0.7373, auc/scab: 0.7547
Evaluating Results - Average Loss: 1.1430 | auc/_mean: 0.7130 | auc/healthy: 0.7608 | auc/multiple_diseases: 0.3905, auc/rust: 0.8392, auc/scab: 0.8615




Epoch: 50 | Time: 0m 14s
Training Results - Average Loss: 1.1936 | auc/_mean: 0.6889 | auc/healthy: 0.6745 | auc/multiple_diseases: 0.5693, auc/rust: 0.7334, auc/scab: 0.7787
Evaluating Results - Average Loss: 1.1397 | auc/_mean: 0.7165 | auc/healthy: 0.7651 | auc/multiple_diseases: 0.3948, auc/rust: 0.8429, auc/scab: 0.8632




In [10]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=50_loss=1.139653613169988.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)
num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

Loaded pretrained weights for efficientnet-b0


In [11]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 21s
Training Results - Average Loss: 1.1924 | auc/_mean: 0.6823 | auc/healthy: 0.6899 | auc/multiple_diseases: 0.5398, auc/rust: 0.7452, auc/scab: 0.7543
Evaluating Results - Average Loss: 1.1400 | auc/_mean: 0.7163 | auc/healthy: 0.7637 | auc/multiple_diseases: 0.3955, auc/rust: 0.8428, auc/scab: 0.8633




Epoch: 02 | Time: 0m 20s
Training Results - Average Loss: 1.1796 | auc/_mean: 0.7020 | auc/healthy: 0.6956 | auc/multiple_diseases: 0.6040, auc/rust: 0.7375, auc/scab: 0.7711
Evaluating Results - Average Loss: 1.1044 | auc/_mean: 0.7417 | auc/healthy: 0.8041 | auc/multiple_diseases: 0.3950, auc/rust: 0.8814, auc/scab: 0.8863




Epoch: 03 | Time: 0m 20s
Training Results - Average Loss: 1.1586 | auc/_mean: 0.7070 | auc/healthy: 0.7313 | auc/multiple_diseases: 0.5084, auc/rust: 0.7920, auc/scab: 0.7962
Evaluating Results - Average Loss: 1.0637 | auc/_mean: 0.7640 | auc/healthy: 0.8418 | auc/multiple_diseases: 0.3929, auc/rust: 0.9147, auc/scab: 0.9065




Epoch: 04 | Time: 0m 20s
Training Results - Average Loss: 1.1202 | auc/_mean: 0.7464 | auc/healthy: 0.7728 | auc/multiple_diseases: 0.5322, auc/rust: 0.8460, auc/scab: 0.8348
Evaluating Results - Average Loss: 1.0184 | auc/_mean: 0.7824 | auc/healthy: 0.8732 | auc/multiple_diseases: 0.3969, auc/rust: 0.9395, auc/scab: 0.9199




Epoch: 05 | Time: 0m 20s
Training Results - Average Loss: 1.0926 | auc/_mean: 0.7697 | auc/healthy: 0.8090 | auc/multiple_diseases: 0.5655, auc/rust: 0.8644, auc/scab: 0.8400
Evaluating Results - Average Loss: 0.9692 | auc/_mean: 0.7982 | auc/healthy: 0.8995 | auc/multiple_diseases: 0.4041, auc/rust: 0.9575, auc/scab: 0.9319



...


Epoch: 156 | Time: 0m 19s
Training Results - Average Loss: 0.4234 | auc/_mean: 0.9960 | auc/healthy: 0.9987 | auc/multiple_diseases: 0.9883, auc/rust: 0.9990, auc/scab: 0.9980
Evaluating Results - Average Loss: 0.4438 | auc/_mean: 0.9728 | auc/healthy: 0.9987 | auc/multiple_diseases: 0.8979, auc/rust: 0.9982, auc/scab: 0.9965




Epoch: 157 | Time: 0m 20s
Training Results - Average Loss: 0.4097 | auc/_mean: 0.9972 | auc/healthy: 0.9997 | auc/multiple_diseases: 0.9918, auc/rust: 0.9988, auc/scab: 0.9986
Evaluating Results - Average Loss: 0.4454 | auc/_mean: 0.9718 | auc/healthy: 0.9988 | auc/multiple_diseases: 0.8940, auc/rust: 0.9982, auc/scab: 0.9963




Epoch: 158 | Time: 0m 20s
Training Results - Average Loss: 0.4212 | auc/_mean: 0.9969 | auc/healthy: 0.9966 | auc/multiple_diseases: 0.9962, auc/rust: 0.9982, auc/scab: 0.9966
Evaluating Results - Average Loss: 0.4441 | auc/_mean: 0.9724 | auc/healthy: 0.9988 | auc/multiple_diseases: 0.8959, auc/rust: 0.9983, auc/scab: 0.9964




Epoch: 159 | Time: 0m 19s
Training Results - Average Loss: 0.4168 | auc/_mean: 0.9950 | auc/healthy: 0.9996 | auc/multiple_diseases: 0.9845, auc/rust: 0.9971, auc/scab: 0.9989
Evaluating Results - Average Loss: 0.4431 | auc/_mean: 0.9724 | auc/healthy: 0.9989 | auc/multiple_diseases: 0.8959, auc/rust: 0.9981, auc/scab: 0.9964




Epoch: 160 | Time: 0m 19s
Training Results - Average Loss: 0.4115 | auc/_mean: 0.9982 | auc/healthy: 0.9986 | auc/multiple_diseases: 0.9962, auc/rust: 0.9992, auc/scab: 0.9989
Evaluating Results - Average Loss: 0.4432 | auc/_mean: 0.9726 | auc/healthy: 0.9989 | auc/multiple_diseases: 0.8966, auc/rust: 0.9982, auc/scab: 0.9966

EarlyStopping


In [11]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [12]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [13]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=125_auc_mean=0.9759388963311403.pth"))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [14]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [15]:
y_probas = np.mean(result, axis=-1)

In [16]:
y_probas[:10]

array([[0.11525081, 0.03328629, 0.794872  , 0.05659093],
       [0.02689193, 0.08945929, 0.86251545, 0.02113327],
       [0.01418664, 0.03105108, 0.02221094, 0.9325514 ],
       [0.9593188 , 0.00964081, 0.01439708, 0.01664325],
       [0.01813011, 0.03230838, 0.92921793, 0.02034356],
       [0.91774046, 0.01537943, 0.03287008, 0.03401009],
       [0.9554141 , 0.01326885, 0.02219261, 0.00912439],
       [0.01802489, 0.20799896, 0.04579664, 0.72817945],
       [0.01979253, 0.5242635 , 0.03942884, 0.4165151 ],
       [0.03681117, 0.08224317, 0.8446248 , 0.03632084]], dtype=float32)

In [17]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.115251,0.033286,0.794872,0.056591
1,Test_1,0.026892,0.089459,0.862515,0.021133
2,Test_2,0.014187,0.031051,0.022211,0.932551
3,Test_3,0.959319,0.009641,0.014397,0.016643
4,Test_4,0.018130,0.032308,0.929218,0.020344


In [18]:
config.submission_file

'02_starter_code.csv'